In [1]:
import numpy as np
import pandas as pd

In [2]:
bb = pd.read_csv("D:/project/homecredit/bureau_balance.csv")

bureau = pd.read_csv("D:/project/homecredit/bureau.csv", usecols=['SK_ID_CURR', 'SK_ID_BUREAU'])


bb.sort_values(by=['SK_ID_BUREAU','MONTHS_BALANCE'],inplace=True)
bureau.sort_values(by=['SK_ID_CURR', 'SK_ID_BUREAU'],inplace=True)

In [3]:
total_status_count = pd.crosstab(bb['SK_ID_BUREAU'], bb['STATUS'],normalize=0)
total_status_count.rename(columns={'0':'total_status_0%','1':'total_status_1%','2':'total_status_2%',
                                '3':'total_status_3%','4':'total_status_4%','5':'total_status_5%',
                                'C':'total_status_C%','X':'total_status_X%'},inplace=True) 
total_status_count.reset_index(level='SK_ID_BUREAU',inplace=True)

In [4]:
bb_g_id = pd.DataFrame(bb.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].max())
bb_g_id.reset_index(level='SK_ID_BUREAU',inplace=True)
bb_g_id.rename(columns={'MONTHS_BALANCE':'MONTHS_BALANC_MAX'},inplace=True)
bb_m = pd.merge(bb,bb_g_id)
bb_m.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,MONTHS_BALANC_MAX
0,5001709,-96,X,0
1,5001709,-95,X,0
2,5001709,-94,X,0
3,5001709,-93,X,0
4,5001709,-92,X,0


In [5]:
bb_m['recent_3'] = (bb_m['MONTHS_BALANCE'] >= bb_m['MONTHS_BALANC_MAX']-2) #新增欄位，並標示MONTHS_BALANCE是否為最近3期
bb_m['recent_6'] = (bb_m['MONTHS_BALANCE'] >= bb_m['MONTHS_BALANC_MAX']-5) #新增欄位，並標示MONTHS_BALANCE是否為最近6期
bb_m['recent_9'] = (bb_m['MONTHS_BALANCE'] >= bb_m['MONTHS_BALANC_MAX']-8) #新增欄位，並標示MONTHS_BALANCE是否為最近9期

recent_3_status=bb_m[bb_m["recent_3"]==True].sort_values(by=["SK_ID_BUREAU"]) #新增表格，篩選最近3期MONTHS_BALANCE
recent_6_status=bb_m[bb_m["recent_6"]==True].sort_values(by=["SK_ID_BUREAU"]) #新增表格，篩選最近6期MONTHS_BALANCE
recent_9_status=bb_m[bb_m["recent_9"]==True].sort_values(by=["SK_ID_BUREAU"]) #新增表格，篩選最近9期MONTHS_BALANCE

recent_3_status=pd.crosstab(recent_3_status['SK_ID_BUREAU'], recent_3_status['STATUS'],normalize=0) #列出最近3期STATUS的比例
recent_6_status=pd.crosstab(recent_6_status['SK_ID_BUREAU'], recent_6_status['STATUS'],normalize=0) #列出最近6期STATUS的比例
recent_9_status=pd.crosstab(recent_9_status['SK_ID_BUREAU'], recent_9_status['STATUS'],normalize=0) #列出最近9期STATUS的比例

recent_3_status.rename(columns={'0':'recent_3_status_0%','1':'recent_3_status_1%','2':'recent_3_status_2%',
                                '3':'recent_3_status_3%','4':'recent_3_status_4%','5':'recent_3_status_5%',
                                'C':'recent_3_status_C%','X':'recent_3_status_X%'},inplace=True)  #更改最近3期欄位名稱
recent_6_status.rename(columns={'0':'recent_6_status_0%','1':'recent_6_status_1%','2':'recent_6_status_2%',
                                '3':'recent_6_status_3%','4':'recent_6_status_4%','5':'recent_6_status_5%',
                                'C':'recent_6_status_C%','X':'recent_6_status_X%'},inplace=True)  #更改最近6期欄位名稱
recent_9_status.rename(columns={'0':'recent_9_status_0%','1':'recent_9_status_1%','2':'recent_9_status_2%',
                                '3':'recent_9_status_3%','4':'recent_9_status_4%','5':'recent_9_status_5%',
                                'C':'recent_9_status_C%','X':'recent_9_status_X%'},inplace=True)  #更改最近9期欄位名稱


In [6]:
recent_369_status=[recent_3_status,recent_6_status,recent_9_status]
recent_369_status=pd.concat(recent_369_status,axis=1)
recent_369_status.reset_index(level='SK_ID_BUREAU',inplace=True)

In [7]:
bb2=pd.merge(total_status_count, recent_369_status)
# bureau_balance_ETL_v2.to_csv('D:/project/homecredit/final/bureau_balance_ETL_v2',index = False)
# bb2.to_csv('bureau_balance_ETL_v2.csv',index = False)

In [8]:
bb_merge = pd.merge(bureau,bb2,how='outer')

In [9]:
dict_type = {}
for i in bb_merge.columns:
        dict_type[i] = ['max','mean']
num_aggregations = dict_type

bureau_balance_ETL_v2 = bb_merge.groupby(['SK_ID_CURR']).agg({**num_aggregations})


# 重新排列欄位
columns = []

for m in bureau_balance_ETL_v2 .columns.levels[0]:
    for n in bureau_balance_ETL_v2 .columns.levels[1]:
        columns.append(f'BU_{m}_{n}')
        
bureau_balance_ETL_v2 .columns = columns

bureau_balance_ETL_v2 .reset_index(level=('SK_ID_CURR'),inplace=True)

In [13]:
bureau_balance_ETL_v2 = bureau_balance_ETL_v2.drop(columns=['BU_SK_ID_CURR_max', 'BU_SK_ID_CURR_mean',
       'BU_SK_ID_BUREAU_max', 'BU_SK_ID_BUREAU_mean'])
bureau_balance_ETL_v2.shape

KeyError: "['BU_SK_ID_CURR_max', 'BU_SK_ID_CURR_mean', 'BU_SK_ID_BUREAU_max', 'BU_SK_ID_BUREAU_mean'] not found in axis"

In [14]:
bureau_balance_ETL_v2.shape

(305811, 65)